<a href="https://colab.research.google.com/github/CE334/CE334_200633_Nikhil_bhardwaj/blob/main/Ce334_Nikhil_Bhardwaj_200633_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Import NumPy for mathematical calculations
import numpy as np

#  WGS84 PARAMETERS
a = 6378137.0            # Semi major axis (meters)
f = 1 / 298.257223563           # Flattaning
e2 = 2*f - f**2             # Eccentricity squared

# JAIPUR COORDINATES
lat_j = 26.9196               # Latitude in degrees
lon_j = 75.7878           # Longitude in degrees
h_j = 435                     # Height in meters

print("Jaipur Coordinates:")
print("Latitude:", lat_j)
print("Longitude:", lon_j)
print("Height:", h_j)


Jaipur Coordinates:
Latitude: 26.9196
Longitude: 75.7878
Height: 435


**PART 1:** Geodetic ↔ Cartesian (ECEF)

In [14]:
def geodetic_to_ecef(lat, lon, h):
       # Converting latitude and longitude to radians
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)

# Computing radius of curvature in prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(lat)**2)

      # Computing Cartesian coordinates
    x = (N + h) * np.cos(lat) * np.cos(lon)
    y = (N + h) * np.cos(lat) * np.sin(lon)
    z = ((1 - e2)*N + h) * np.sin(lat)

    return x, y, z


# Computing ECEF of Jaipur
xj, yj, zj = geodetic_to_ecef(lat_j, lon_j, h_j)

print("ECEF Coordinates of Jaipur:")
print("x =", xj)
print("y =", yj)
print("z =", zj)


ECEF Coordinates of Jaipur:
x = 1397295.9065516565
y = 5517119.519079032
z = 2870472.105193852


ECEF → Geodetic (Iterative)

In [21]:
def ecef_to_geodetic(x, y, z, tol=1e-12):
      # Longitude from x and y
    lon = np.arctan2(y, x)

     # Distance from Z-axis
    p = np.sqrt(x**2 + y**2)

           # Initial latitude estimate
    lat = np.arctan2(z, p*(1 - e2))

    # Iterative improvement
    while True:
        N = a / np.sqrt(1 - e2*np.sin(lat)**2)
        h = p/np.cos(lat) - N
        lat_new = np.arctan2(z, p*(1 - (N*e2)/(N + h)))
        if abs(lat - lat_new) < tol:
            break
        lat = lat_new

    return np.rad2deg(lat), np.rad2deg(lon), h


  # Verify by converting back
lat2, lon2, h2 = ecef_to_geodetic(xj, yj, zj)

print("Back Converted Geodetic Coordinates:")
print("Latitude =", lat2)
print("Longitude =", lon2)
print("Height =", h2)


Back Converted Geodetic Coordinates:
Latitude = 26.919600000020054
Longitude = 75.7878
Height = 435.00000112876296


**PART 2:** Spherical ↔ Cartesian

In [22]:
def spherical_to_cartesian(phi, lam, r):
   #  Now Converting degrees to radians
    phi = np.deg2rad(phi)
    lam = np.deg2rad(lam)

    # Spherical to Cartesian formula
    x = r * np.cos(phi) * np.cos(lam)
    y = r * np.cos(phi) * np.sin(lam)
    z = r * np.sin(phi)

    return x, y, z


# Assume Earth radius = semi-major axis
r = a

xs, ys, zs = spherical_to_cartesian(lat_j, lon_j, r)

print("Spherical to Cartesian:")
print(xs, ys, zs)


Spherical to Cartesian:
1396241.7536128398 5512957.274183322 2887636.169822358


cartesian to Spherical

In [17]:
def cartesian_to_spherical(x, y, z):
    # Radius
    r = np.sqrt(x**2 + y**2 + z**2)

    # Latitude
    phi = np.arcsin(z / r)

    # Longitude
    lam = np.arctan2(y, x)

    return np.rad2deg(phi), np.rad2deg(lam), r


phi_s, lam_s, r_s = cartesian_to_spherical(xs, ys, zs)

print("Back to Spherical:")
print("Latitude =", phi_s)
print("Longitude =", lam_s)
print("Radius =", r_s)


Back to Spherical:
Latitude = 26.9196
Longitude = 75.7878
Radius = 6378137.0


**Part 3:** Geodetic to Spherical (Using Cartesian)

In [27]:
def geodetic_to_spherical_ref(lat, lon, h, a, e2, r_sphere=None):

    # lat, lon, h   : Geodetic coordinates (degrees, degrees, meters)
    # a, e2      : Reference ellipsoid parameters
    # r_sphere      : Radius of reference sphere (optional)
    # If r_sphere is not given, it is taken as semi-major axis 'a'


    # use ellipsoid semi-major axis
    if r_sphere is None:
        r_sphere = a

    #  Geodetic to Cartesian (ECEF)
    lat_rad = np.deg2rad(lat)
    lon_rad = np.deg2rad(lon)

    # Radius of curvature in prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(lat_rad)**2)

    # ECEF coordinates using ellipsoidal model
    x = (N + h) * np.cos(lat_rad) * np.cos(lon_rad)
    y = (N + h) * np.cos(lat_rad) * np.sin(lon_rad)
    z = ((1 - e2)*N + h) * np.sin(lat_rad)
    # Cartesian to Spherical
    # Radius from Earth's center
    r = np.sqrt(x**2 + y**2 + z**2)

    # Spherical latitude
    phi = np.arcsin(z / r)

    # Spherical longitude
    lam = np.arctan2(y, x)

    # Convert angles to degrees
    return np.rad2deg(phi), np.rad2deg(lam), r


In [28]:
phi_s, lam_s, r_s = geodetic_to_spherical_ref(lat_j, lon_j, h_j, a, e2)

print("Geodetic to Spherical (Sphere Radius = a):")
print("Latitude =", phi_s)
print("Longitude =", lam_s)
print("Radius =", r_s)


Geodetic to Spherical (Sphere Radius = a):
Latitude = 26.764562013482152
Longitude = 75.7878
Radius = 6374217.892805743


In [29]:
#  provide different sphere radius (mean Earth radius)
r_mean = 6371000   # meters

phi_s2, lam_s2, r_s2 = geodetic_to_spherical_ref(lat_j, lon_j, h_j, a, e2, r_mean)

print("Geodetic to Spherical ( Defined Sphere Radius):")
print("Latitude =", phi_s2)
print("Longitude =", lam_s2)
print("Radius =", r_s2)


Geodetic to Spherical ( Defined Sphere Radius):
Latitude = 26.764562013482152
Longitude = 75.7878
Radius = 6374217.892805743


Topocentric Coordinates (ENU) Satellite Directly Over Jaipur (800 km)

In [30]:
def ecef_to_enu(dx, dy, dz, lat, lon):
    # Convert to radians
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)

    # Rotation matrix
    R = np.array([
        [-np.sin(lon),              np.cos(lon),             0],
        [-np.cos(lon)*np.sin(lat), -np.sin(lon)*np.sin(lat), np.cos(lat)],
        [ np.cos(lon)*np.cos(lat),  np.sin(lon)*np.cos(lat), np.sin(lat)]
    ])

    d = np.array([dx, dy, dz])
    enu = R @ d

    return enu[0], enu[1], enu[2]


# Satellite height
hs = h_j + 800000

# Satellite ECEF
xsat, ysat, zsat = geodetic_to_ecef(lat_j, lon_j, hs)

# Displacement from Jaipur
dx = xsat - xj
dy = ysat - yj
dz = zsat - zj

# Convert to ENU
e, n, u = ecef_to_enu(dx, dy, dz, lat_j, lon_j)

print("Topocentric Coordinates of Satellite:")
print("East =", e)
print("North =", n)
print("Up =", u)


Topocentric Coordinates of Satellite:
East = -2.0990795276825662e-10
North = -1.7344299506688725e-10
Up = 800000.0000000005
